In [1]:
import pandas as pd
import re

In [2]:
## Can be downloaded from COSMIC
mutationfile = pd.read_csv("/Users/davidshorthouse/Documents/Data/COSMIC/Large_intestine_COSMIC_mutations.csv")

/var/folders/vv/0521912d5_j4wpw2lwprbrzw0000gn/T/ipykernel_13724/766392041.py:2: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  mutationfile = pd.read_csv("/Users/davidshorthouse/Documents/Data/COSMIC/Large_intestine_COSMIC_mutations.csv")


In [3]:
mutationfile = mutationfile.drop_duplicates(subset = ["COSMIC_SAMPLE_ID", "GENOMIC_MUTATION_ID"])

In [4]:
mutationfile.head()

,Unnamed: 0,GENE_SYMBOL,COSMIC_GENE_ID,TRANSCRIPT_ACCESSION,COSMIC_SAMPLE_ID,SAMPLE_NAME,COSMIC_PHENOTYPE_ID,GENOMIC_MUTATION_ID,LEGACY_MUTATION_ID,MUTATION_ID,...,GENOME_STOP,STRAND,PUBMED_PMID,COSMIC_STUDY_ID,HGVSP,HGVSC,HGVSG,GENOMIC_WT_ALLELE,GENOMIC_MUT_ALLELE,MUTATION_SOMATIC_STATUS
0,14,PGAM5,COSG3464,ENST00000454808.2,COSS2607147,HUB-02-B2-070,COSO28864826,COSV58203115,COSM6040759,61070074,...,133295382.0,+,NaN,COSU670,NaN,ENST00000454808.2:c.272+676C>T,12:g.133295382C>T,C,T,Confirmed somatic variant
1,31,PTPN1,COSG8144,ENST00000371621.3,COSS1651176,TCGA-AZ-6603-01,COSO28694826,COSV50403403,COSM1412424,38682688,...,49197963.0,+,NaN,COSU376,ENSP00000360683.3:p.Thr417Met,ENST00000371621.3:c.1250C>T,20:g.49197963C>T,C,T,Confirmed somatic variant
2,43,ATXN2L,COSG46295,ENST00000395547.2,COSS1651197,TCGA-CK-5913-01,COSO28454826,COSV100294844,COSM7605644,47064055,...,28847651.0,+,NaN,COSU376,ENSP00000378917.2:p.Cys1048=,ENST00000395547.2:c.3144T>C,16:g.28847651T>C,T,C,Confirmed somatic variant
3,51,ARHGEF10,COSG21039,ENST00000518288.1,COSS2506609,sysucc-1422T,COSO28864826,COSV50643338,COSN20458451,78787872,...,1807346.0,+,NaN,COSU646,NaN,ENST00000518288.1:c.266-717C>G,8:g.1807346C>G,C,G,Confirmed somatic variant
4,68,LINGO2,COSG51151,ENST00000308675.3,COSS2658290,T3319,COSO28864826,COSV58058127,COSM6701436,26721300,...,27950274.0,-,27149842.0,NaN,ENSP00000310126.3:p.Thr132=,ENST00000308675.3:c.396T>C,9:g.27950274A>G,A,G,Confirmed somatic variant


In [5]:
WNT5A = mutationfile[mutationfile["GENE_SYMBOL"] == "WNT5A"]
WNT5A = WNT5A[WNT5A["MUTATION_DESCRIPTION"] == "missense_variant"]

In [6]:
def makemutantlist(df):

    mutationlist = []
    for line in df["MUTATION_AA"]:
        if line.startswith("p."):
            mutation = line.split("p.")[1]
            if mutation == "?":
                continue
            elif "*" in mutation:
                continue
            else:
                startaa = mutation[0]
                endaa = mutation[-1]
                if startaa == endaa:
                    continue
                else:
                    mutantnumber = int(re.findall('\d+', mutation)[0])
                    mutationlist.append(mutantnumber)
    return(mutationlist)


In [7]:
genename = "WNT3"
proteinlength = 355


gene_only_df = mutationfile[mutationfile["GENE_SYMBOL"] == genename]
gene_only_df = gene_only_df.drop_duplicates()

gene_only_df = gene_only_df[gene_only_df["MUTATION_DESCRIPTION"] == "missense_variant"]

mutationlist = makemutantlist(gene_only_df)

listsforarray = []
listsforarray.append(list(range(1, proteinlength+1)))
for item in mutationlist:
    zerolist = [0] * proteinlength
    zerolist[item-1] = 1
    listsforarray.append(zerolist)
mutationdataframe = pd.DataFrame(listsforarray)
mutationdataframe.columns = mutationdataframe.iloc[0]
mutationdataframe = mutationdataframe.reindex(mutationdataframe.index.drop(0))



In [8]:
def makematrix(genename, proteinlength):
    gene_only_df = mutationfile[mutationfile["GENE_SYMBOL"] == genename]
    gene_only_df = gene_only_df.drop_duplicates()

    gene_only_df = gene_only_df[gene_only_df["MUTATION_DESCRIPTION"] == "missense_variant"]

    mutationlist = makemutantlist(gene_only_df)

    listsforarray = []
    listsforarray.append(list(range(1, proteinlength+1)))
    for item in mutationlist:
        zerolist = [0] * proteinlength
        zerolist[item-1] = 1
        listsforarray.append(zerolist)
    mutationdataframe = pd.DataFrame(listsforarray)
    mutationdataframe.columns = mutationdataframe.iloc[0]
    mutationdataframe = mutationdataframe.reindex(mutationdataframe.index.drop(0))

    mutationdataframe.to_csv(genename + "_mutationmatrix_NMCinput.csv", index= False)

In [9]:
makematrix("WNT5A", 380)